# Reaction annotation
Using MetaNetX reac_xref.tsv (can  be downloaded from [here](https://www.metanetx.org/mnxdoc/mnxref.html)).

In [1]:
import re
from collections import defaultdict
from pathlib import Path

import cobra
from datatable import dt, f, join, update

In [2]:
ROOT = Path.cwd().parent
model_file = str(ROOT / "iMENI452.xml")

In [3]:
model = cobra.io.read_sbml_model(model_file)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Gather all prepared reactions in a dataframe matched to their identifiers in the model.

In [4]:
reacs_prepared = [reac.id for reac in model.reactions]

In [5]:
reacs_prepared[:5]

['ALAD_L', 'ASPTA', 'ARGSS', 'ARGSL', 'ADSS']

Compartments in the reactions (mostly exchanges) should be "\_e" instead of "\[e\]" 

In [6]:
REAC_PAT = re.compile(r"\[([ce])\]$")
UNDER_PAT = re.compile(r"-([CBT])")  # acon-C should be acon_C
DUNDER_PAT = re.compile(r"-([DLSR])")  # ala-L shoul´d be ala__L

In [7]:
reacs_prepared = [REAC_PAT.sub(r"_\1", reac) for reac in reacs_prepared]

In [8]:
reacs_prepared = [
    UNDER_PAT.sub(r"_\1", DUNDER_PAT.sub(r"__\1", reac))
    if reac.startswith("EX_")
    else reac
    for reac in reacs_prepared
]

These are bigg reactions which can be already annotated.

The identifiers won't be changed right now to keep compatible with inhouse scripts, but this will have some slight impact in the score (all identfiers should be consistent with the way the are written in the largest found database/namespace).

In [9]:
for reac, annot in zip(model.reactions, reacs_prepared):
    reac.annotation["bigg.reaction"] = annot

In [10]:
reacs_prepared = dt.Frame(
    key=reacs_prepared, in_model=[reac.id for reac in model.reactions]
)

In [11]:
reacs_prepared.head()

<Frame#7f3d3021f6c0 10x2>

Now that we have the model reactions in a data frame, let's prepare the MNX dataset.

In [12]:
mnx = dt.fread(str(ROOT / "reac_xref.tsv"), skip_to_line=352)

In [13]:
mnx.head()

<Frame#7f3cea285120 10x3>

In [14]:
mnx.tail()

<Frame#7f3cea28ce70 10x3>

In [15]:
mnx.names = ["key", "mnx", "description"]

## MNX space

The first step is to map the BiGG reaction identifiers to MXN.

In [16]:
bigg_mnx = mnx[
    dt.re.match(f.key, "^bigg\.reaction:.+"), {"key": f.key[14:], "mnx": f.mnx}
]

In [17]:
bigg_mnx[dt.re.match(f.key, "EX_glyc__R.*"), :]

<Frame#7f3cea2d4150 1x2>

In [18]:
bigg_mnx.key = "key"

Join by bigg id in the model, so that we take only the reactions of interest.

In [19]:
reacs_prepared = reacs_prepared[:, :, join(bigg_mnx)]

In [20]:
reacs_prepared.nrows

813

In [21]:
reacs_prepared[dt.isna(f.mnx), :].nrows

182

In [22]:
reacs_prepared[dt.isna(f.mnx), :]

<Frame#7f3cea2d4840 182x3>

In [23]:
len(model.reactions)

813

In [24]:
for reac in model.reactions:
    matched = reacs_prepared[f.in_model == reac.id, "mnx"]
    if matched.nrows:
        annot = matched[0, 0]
        if matched:
            reac.annotation["metanetx.reaction"] = matched[0, 0]

In [25]:
len(
    [
        1
        for reac in model.reactions
        if "metanetx.reaction" in reac.annotation
        if reac.annotation["metanetx.reaction"] is not None
    ]
)

631

In [26]:
for reac in model.reactions:
    if "metanetx.reaction" in reac.annotation:
        if reac.annotation["metanetx.reaction"] is None:
            del reac.annotation["metanetx.reaction"]

In [27]:
len(model.reactions)

813

## Annotating BiGG

In [28]:
mnx["db"] = "mnx"

In [29]:
def get_id_db(id_base: str) -> (str, str):
    id_base = id_base.split(":", maxsplit=1)
    if len(id_base) > 1:
        db, id = id_base
    else:
        db, id = "mnx", id_base[0]
    return id, db

In [30]:
ids_dbs = [get_id_db(mnx[i, "key"]) for i in range(mnx.nrows)]
ids = [el[0] for el in ids_dbs]
dbs = [el[1] for el in ids_dbs]
mnx["key"] = dt.Frame(ids)
mnx["db"] = dt.Frame(dbs)

In [33]:
for reac in model.reactions:
    if "metanetx.reaction" in reac.annotation:
        kv = mnx[f.mnx == reac.annotation["metanetx.reaction"], ["key", "db"]].to_list()
        data_dict = defaultdict(list)
        for k, v in zip(kv[1], kv[0]):
            data_dict[k].append(v)
        if reac.annotation["metanetx.reaction"]:
            data_dict["metanetx.reaction"] = reac.annotation["metanetx.reaction"]
        for k, v in data_dict.items():
            data_dict[k] = v if len(v) > 1 else v[0]
        reac.annotation = data_dict

In [34]:
dbs = dt.unique(mnx["db"]).to_list()[0]

In [35]:
dbs

['bigg.reaction',
 'biggR',
 'kegg.reaction',
 'keggR',
 'metacyc.reaction',
 'metacycR',
 'mnx',
 'rhea',
 'rheaR',
 'sabiork.reaction',
 'sabiorkR',
 'seed.reaction',
 'seedR']

In [36]:
for db in dbs:
    print(
        f"{db} annotated: {len([1 for reac in model.reactions if db in reac.annotation])}"
    )

bigg.reaction annotated: 813
biggR annotated: 631
kegg.reaction annotated: 286
keggR annotated: 286
metacyc.reaction annotated: 332
metacycR annotated: 332
mnx annotated: 1
rhea annotated: 321
rheaR annotated: 321
sabiork.reaction annotated: 253
sabiorkR annotated: 253
seed.reaction annotated: 483
seedR annotated: 483


"metanetx.reaction" annotations were removed while computing the others. Add them back

In [37]:
len([1 for reac in model.reactions if "metanetx.reaction" in reac.annotation])

631

In [38]:
for reac in model.reactions:
    if reac.annotation is None:
        reac.annotation = {}
    to_del = [k for k, v in reac.annotation.items() if v is None]
    for k in to_del:
        del reac.annotation[k]
        del reac.annotation["mnx"]

In [39]:
cobra.io.write_sbml_model(model, model_file)

## SBO terms

SBO:0000176 represents the term 'biochemical reaction'. Every metabolic reaction that is not a transport or boundary reaction should be annotated with this.

In [40]:
from memote.support.helpers import find_transport_reactions

In [41]:
transport_reactions = find_transport_reactions(model)

In [42]:
for reac in model.reactions:
    reac.annotation["sbo"] = (
        "SBO:0000185" if reac in transport_reactions else "SBO:0000176"
    )

Now, biomass reaction.

In [43]:
model.reactions.biomass.annotation["sbo"] = "SBO:0000629"

SBO:0000627 represents the term 'exchange reaction'.

In [44]:
for ex in model.exchanges:
    ex.annotation["sbo"] = "SBO:0000627"

SBO:0000243 represents the term 'gene'. Every gene should be annotated with this.

In [45]:
for gene in model.genes:
    gene.annotation["sbo"] = "SBO:0000243"

In [46]:
cobra.io.write_sbml_model(model, model_file)